## Question 2 EM

Yilin Gao (yg95)

In [1]:
import numpy as np

### simulate data

m = 6, n = 100,

for each m, 50% chance using machine 1, and 50% change using machine 2

theta1 = 0.8, theta2 = 0.3

1: broken, 0: not broken

In [85]:
X = np.empty([0, 100])
for i in range(6):
    machine = np.random.rand() # random float in [0, 1) with uniform distribution
    broken_prop = 0
    if machine > 0.5: # machine 2
        broken_prop = 0.3
    else: # machine 1
        broken_prop = 0.8
    row = np.random.rand(1, 100) # random float in [0, 1) with uniform distribution
    broken_index = row < broken_prop
    row[0, :] = 0
    row[broken_index] = 1
    X = np.append(X, row, axis=0)

In [86]:
print(X.shape)
broken_count = np.sum(X, axis=1)
print(broken_count)

(6, 100)
[33. 35. 81. 71. 75. 81.]


### initialize parameters

In [80]:
w = np.asarray([0.5, 0.5])
theta = np.asarray([0.9, 0.1])

### run EM algorithm

In [38]:
def has_converged(a, b):
    '''
    tell if 2 numpy arrays are element-wise equal within a tolerance
    '''
    if (a is None or a is None):
        return False
    assert a.shape[0] == b.shape[0]
    # Returns True if two arrays are element-wise equal within a tolerance.
    return np.allclose(a, b)

In [79]:
def EM_onestep(X, w, theta):
    m, n = X.shape
    assert w.shape[0] == theta.shape[0]
    k = w.shape[0]
    ## E step
    gamma = np.zeros([m, k])
    for i in range(k):
        gamma[:, i] = np.prod((theta[i] ** X) * ((1 - theta[i]) ** (1 - X)), axis=1) * w[i]
    gamma_denominator = np.sum(gamma, axis=1).reshape((m, 1))
    gamma = gamma / gamma_denominator
    ## M step
    theta_new = np.zeros(2)
    w_new = np.zeros(2)
    for i in range(k):
        theta_denominator = np.sum(gamma[:, i])
        theta_nominator = np.dot(gamma[:, i], np.sum(X, axis=1) / n)
        theta_new[i] = theta_nominator / theta_denominator
        w_new[i] = theta_denominator / m
    return w_new, theta_new

In [88]:
print(EM_onestep(X, w, theta))

(array([0.66666667, 0.33333333]), array([0.77, 0.34]))


In [78]:
def EM(X, w, theta):
    w_old, theta_old = None, None
    w_new, theta_new = w, theta
    while (not has_converged(w_old, w_new) and not has_converged(theta_old, theta_new)):
        w_old, theta_old = w_new, theta_new
        w_new, theta_new = EM_onestep(X, w_old, theta_old)
    return w_new, theta_new

In [87]:
print(EM(X, w, theta))

(array([0.66666667, 0.33333333]), array([0.77, 0.34]))
